## 오늘 해볼 것은 MNIST 

하나의 숫자 이미지는 28x28  
이것을 1차원으로 주욱 펴서 784짜리 1차원 벡터로 즉, 784개의 특성을 가진 것으로 만들어  


In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

In [ ]:
X, y  = mnist["data"], mnist["target"]
print(X.shape, y.shape)

In [ ]:
type(X)

In [ ]:
import matplotlib.pyplot as plt
some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()

binary를 써주면 0에 가까울수록 하얀색 255에 가까울수록 검정색

In [ ]:
plt.imshow(some_digit_image, cmap='gray')
plt.axis("off")
plt.show()

In [ ]:
y[0]

In [ ]:
import numpy as np
y = y.astype(np.uint8) #부호를 빼주면 0~255가 되기 때문에 딱 좋음

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

이진 분류기   
5냐 아니냐로

In [ ]:
y_train_5 = (y_train==5)
y_test_5 = (y_test==5)

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state = 42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
sgd_clf.score(X_train, y_train_5)
sgd_clf.score(X_test, y_test_5)

### 성능 측정

k-겹 교차 검증  
테스트 세트를 k개로 쪼개서 k개 중 하나는 검증세트로 둠

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

결과는 위의 점수들을 평균낸 것

In [ ]:
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
  def fit(self, X, y=None):
    pass
  def predict(self, X):
    return np.zeros((len(X),1),dtype=bool)

In [ ]:
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

# 오차 행렬  
오늘 가장 중요한 분류기 성능 판별

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

NameError: ignored

accuracy = 53892+3530/60000

y=train_perfect_predict = y_train_5
confusion_matrix(y_train_5, y_train_5, 

또다른 평가지표 정밀도 precision  
양성 예측의 정확도  
precision = TP/TP+FP

재현율 recall  
recall = TP/TP+FN

# F1 점수  
정밀도와 재현률의 조화 평균  
산술평균 = a+b/2  
조화평균 = 2ab/a+b  
기하평균 = sqrt(a^2+b^2)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_5, y_train_pred)

정밀도와 재현율 사이에는 트레이드 오프가 일어남  
z=f(x), y = 1/1+e^-z  
일반적으로 분류기는 결정 함수를 사용하여 각 샘플의 점수를 계산하고,  이 점수가 임곗값보다 크면 샘플을 양성 클래스로 예측하고 그렇지 않으면 음성 클래스로 예측함

재현율: 양성중에 진짜 양성  
정밀도: 양성이라 예측한 것 중에 진짜 양성인 것들

ROC curve  
TPR 민감도(재현율) 
TNR 특이도   
거짓 양성 비율FPR = 1-TNR  

ROC는 1-특이도에 대한 민감도 곡선

In [ ]:
from sklearn.metrics import roc_curve


곡선 아래(auc)의 면적이 1에 가까울 수록 좋은 분류기다!

## 이진 분류기를 이용한 다중 클래스 분류 

전략1. OvR one versus rest 전략  

전략2. OvO one versus one 전략  

이것도 Trade-off 고려해서 결정 SVM은 OvO

SVC의 기본값은 OVR